In [122]:
from sklearn import tree # módulo de arvore de decisao
from sklearn import preprocessing # Modulo de preprocessamento
from sklearn import ensemble
from sklearn import metrics
import pandas as pd # Pandas manipulação de dados
import os # acesso ao sistema operacional
import graphviz
import numpy as np

In [47]:
df = pd.read_csv( 'dia05/dados_candidatos.csv', sep=';', decimal=',' )

target = 'descricao_grau_instrucao'
features = [ 'descricao_cargo', 'descricao_cor_raca', 'descricao_estado_civil', 'descricao_genero',
                'sigla_uf_nascimento', 'nome_partido', 'patrimonio'  ]

group_names = { 'LE E ESCREVE': 'SEM ESCOLARIDADE',
                'ENSINO FUNDAMENTAL INCOMPLETO': 'SEM ESCOLARIDADE',
                'ENSINO MEDIO INCOMPLETO':'ENSINO FUNDAMENTAL COMPLETO',
                'SUPERIOR INCOMPLETO': 'ENSINO MEDIO COMPLETO' }

df[target] = df[target].apply( lambda x: group_names[x] if x in group_names else x )

# Agrupa suplentes
df['descricao_cargo'] = df['descricao_cargo'].replace( ['1o SUPLENTE SENADOR', '2o SUPLENTE SENADOR'], 'SUPLENTE SENADOR' )

In [55]:
cat_features = df[features].dtypes[df[features].dtypes == 'object'].index
onehot = preprocessing.OneHotEncoder(sparse=False)
onehot.fit( df[cat_features ] )
X = pd.DataFrame( onehot.transform( df[cat_features] ), columns = onehot.get_feature_names( cat_features ) )

num_features = df[features].dtypes[df[features].dtypes != 'object'].index
X[num_features] = df[num_features].copy()
X['declara_patrimonio'] = X['patrimonio'] > 0

In [88]:
df['sample1'] = np.random.random( df.shape[0] ) < 0.6
clf_1 = tree.DecisionTreeClassifier(max_depth=10, min_samples_leaf=20)
clf_1.fit( X[ df['sample1'] ] , df[target][df['sample1']] )

df['sample2'] = np.random.random( df.shape[0] ) < 0.6
clf_2 = tree.DecisionTreeClassifier(max_depth=10, min_samples_leaf=20)
clf_2.fit( X[ df['sample2'] ] , df[target][df['sample2']] )

predict_full = clf.predict( X.iloc[0:3] )
predict_1 = clf_1.predict( X.iloc[0:3] )
predict_2 = clf_2.predict( X.iloc[0:3] )

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=10,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=20, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')

# Definindo base de treino e base de teste

In [ ]:
X_train, X_test, y_train, y_test = model_select

In [119]:
clf_rf = ensemble.RandomForestClassifier( n_estimators=500, max_depth=20, n_jobs=3 )
clf_rf.fit( X, df[target] )

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=20, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=500, n_jobs=3,
                       oob_score=False, random_state=None, verbose=0,
                       warm_start=False)

In [118]:
print(" Acurácia para random forest:", metrics.accuracy_score(  df[target] , clf_rf.predict(X) ) )
print(" Acurácia para Arvore de decisao:", metrics.accuracy_score(  df[target] , clf.predict(X) ) )

Acurácia para random forest: 0.7170706549767921
 Acurácia para Arvore de decisao: 0.7247722193570569
